# Тестовое задание.
 
## Оглавление:

1. Цель работы
2. Задачи
3. Описание имеюющихся данных
4. Ссылка на дашборд в Yandex Data Lens


### 1. Цель работы

Создать дашборд с визуализацией среднего времени ответа на сообщение для каждого менеджера с фильтрацией по дням, менеджерам и начальникам отделов.

Для достижения поставленной цели требуется выполнить следующие задачи:

#### Задачи:

1. Подключение к базе данных
    - Получим необходимые таблицы (test.chat_messages, test.managers, test.rops)
2. Предобработка данных
    - Исследуем таблицы на наличие пропусков/дубликатов
3. Расчет среднего времени ответа каждого менеджера
4. Сохраняем полученные данные в csv-файл
5. Создание дашборда

Описание имеющихся данных:

I. Таблица `test.chat_messages`, содержит информацию о входящих и исходящих сообщениях внутри сделок:
- `message_id` - уникальный идентификатор сообщения;
- `type` -  тип сообщения:
    - `outgoing_chat_message` – исходящее сообщение от менеджера клиенту;
	- `incoming_chat_message` – входящее сообщение от клиента менеджеру;
- `entity_id` - идентификатор сделки в amoCRM. На каждого клиента открывается отдельная сделка;
- `created_by` - идентификатор написавшего сообщение, у клиента всегда 0;
- `created_at` - время создания сообщения в формате Unix Timestamp.

II. Таблица `test.managers`, данные о менеджерах:
- `mop_id` - уникальный идентификатор менеджера;
- `name_mop` - имя менеджера или пары менеджеров;
- `rop_id` - уникальный идентифакатор руководителя отдела продаж.

III. Таблица `test.rops`, содержит данные о руководителях отдела продаж:
- `rop_id` - идентификатор руководителя отдела;
- `rop_name` - имя руководителя.

Приступим к решению первой задачи.

## Подключение к базе данных:

Загрузим все необходимые нам библиотеки. 

Пользоваться будем библиотеками `pandas`, `psycopg2` и модулем `datetime`.

In [1]:
import pandas as pd
import psycopg2
from datetime import datetime, time, timedelta

С помощью библиотеки `psycopg2` подключимся к базе данных и загрузим все таблицы в переменные: `df_chat` - test.chat_messages, `df_managers` - test.managers, `df_rops` - test.rops.

Загрузим их с помощью SQL query запросов.

In [ ]:
# подключаемся к базе данных 
pass_code = psycopg2.connect(
	host="host",
	user="user",
	password="pass",
	database="db",
	port=port,
	sslmode="requiered"
)

# загружаем таблицы и сохраняем в переменных
# таблица с чатами
query_chat = """
SELECT *
FROM test.chat_messages
"""
df_chat = pd.read_sql(query_chat, pass_code)

# таблица с менеджерами
query_managers = """
SELECT *
FROM test.managers
"""
df_managers = pd.read_sql(query_managers, pass_code)

# таблица с руководителями
query_rops = """
SELECT *
FROM test.rops
"""
df_rops = pd.read_sql(query_rops, pass_code)

C:\Users\0_0\AppData\Local\Temp\ipykernel_31896\116082644.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_chat = pd.read_sql(query_chat, pass_code)
C:\Users\0_0\AppData\Local\Temp\ipykernel_31896\116082644.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_managers = pd.read_sql(query_managers, pass_code)
C:\Users\0_0\AppData\Local\Temp\ipykernel_31896\116082644.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_rops = pd.read_sql(query_rops, pass_code)


Мы загрузили необходимые нам таблицы, можно переходить к исследованию таблиц и предобработке данных.

## Предобработка данных.

In [3]:
# общая информация о таблице с сообщениями
df_chat.info()
print(f'Количество пропусков в данных = {df_chat.isna().sum()}')
print(f'Количество явных дубликатов в данных = {df_chat.duplicated().sum()}')
df_chat.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18929 entries, 0 to 18928
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   message_id  18929 non-null  object
 1   type        18929 non-null  object
 2   entity_id   18929 non-null  int64 
 3   created_by  18929 non-null  int64 
 4   created_at  18929 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 739.5+ KB
Количество пропусков в данных = message_id    0
type          0
entity_id     0
created_by    0
created_at    0
dtype: int64
Количество явных дубликатов в данных = 0


,message_id,type,entity_id,created_by,created_at
0,"""01jb7da570sf4f65xdf0ptvv9q""",incoming_chat_message,37556493,0,1730046924
1,"""01jb3t6bc8gvyfnd063nd91ng4""",incoming_chat_message,37549491,0,1729926213
2,"""01jb6gj9ngwk0ybbmg9w90pbqg""",incoming_chat_message,37531455,0,1730016782
3,"""01jb75bkprk0d7hht1g5vy1pm4""",incoming_chat_message,37553371,0,1730038583
4,"""01jb3t5xprbph1c6pym8ervxpm""",incoming_chat_message,37548675,0,1729926199


In [4]:
# общая информация о таблице с менеджерами
df_managers.info()
print(f'Количество пропусков в данных = {df_managers.isna().sum()}')
print(f'КОличество явных дубликатов в данных = {df_managers.duplicated().sum()}')
df_managers.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   mop_id    26 non-null     int64 
 1   name_mop  26 non-null     object
 2   rop_id    26 non-null     object
dtypes: int64(1), object(2)
memory usage: 756.0+ bytes
Количество пропусков в данных = mop_id      0
name_mop    0
rop_id      0
dtype: int64
КОличество явных дубликатов в данных = 0


,mop_id,name_mop,rop_id
0,6645315,Гюнель и Илина,1
1,6744792,Юля и Наташа,1
2,10262505,Вика и Марго,1
3,10262513,Ира и Варя,1
4,10465254,Настя и Даша,1


In [5]:
# общая информация о таблице руководителями отделов
df_rops.info()
print(f'Количество пропусков в данных = {df_rops.isna().sum()}')
print(f'КОличество явных дубликатов в данных = {df_rops.duplicated().sum()}')
df_rops.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   rop_id    3 non-null      int64 
 1   rop_name  3 non-null      object
dtypes: int64(1), object(1)
memory usage: 180.0+ bytes
Количество пропусков в данных = rop_id      0
rop_name    0
dtype: int64
КОличество явных дубликатов в данных = 0


,rop_id,rop_name
0,1,Катя РОП
1,2,Полина РОП
2,3,Эля РОП


Явных дубликатов и пропусков в столбцах нет, также отсутствуют ошибки в типах данных столбцов.

Понадеимся на "чистоту" наших данных и перейдем к следующему шагу.

## Расчет среднего времени ответа для каждого менеджера

In [6]:
# прежде чем приступем к созданию новых столбцов 
# создадим функцию для расчета среднего времени ответа

def calculate_work_time(start_ts, end_ts):
    """
    Рассчитывает рабочее время в секундах между двумя timestamp с учётом графика:
    - Рабочие часы: с 09:30 до 00:00 следующего дня
    - Ночные и нерабочие часы исключаются из расчёта
    
    Логика работы:
    1. Если клиент написал в нерабочее время, отсчёт начинается с 09:30
    2. Если ответ дан в нерабочее время, отсчёт заканчивается в 00:00
    3. Для периодов, затрагивающих несколько дней, суммируется рабочее время каждого дня
    """
    # Преобразуем UNIX timestamp в объекты datetime
    start = datetime.fromtimestamp(start_ts)
    end = datetime.fromtimestamp(end_ts)

    # Некорректные временные метки
    if start >= end:
        return 0

    # Корректировка времени начала
    if start.time() < time(9, 30):
        # Если сообщение пришло до начала рабочего дня, 
        # отсчёт начинаем с 09:30 текущего дня
        adjusted_start = datetime.combine(start.date(), time(9, 30))
    else:
        # Для сообщений после 09:30 используем фактическое время
        adjusted_start = start

    # Корректировка времени окончания
    if end.time() >= time(9, 30):
        # Если ответ дан в рабочее время, используем фактическое время
        adjusted_end = end
    else:
        # Если ответ дан ночью, считаем до 09:30 текущего дня
        # (не следующего, так рабочий день формально продолжается до 00:00)
        adjusted_end = datetime.combine(end.date(), time(9, 30))

    # Проверка после корректировок
    if adjusted_start >= adjusted_end:
        return 0

    total_time = timedelta()  # Общее рабочее время
    current_day = adjusted_start.date()  # Текущий день в итерации

    # Итерируем по всем дням в периоде
    while current_day <= adjusted_end.date():
        # Начало рабочего времени для текущего дня
        day_start = max(
            adjusted_start,  # Оригинальное время или 09:30 если было корректировка
            datetime.combine(current_day, time(9, 30))  # 09:30 текущего дня
        )
        
        # Конец рабочего времени для текущего дня
        day_end = min(
            adjusted_end,  # Оригинальное время или 09:30 если было корректировка
            datetime.combine(current_day, time(23, 59, 59))  # До полуночи
        )

        # Добавляем время, только если начало раньше конца
        if day_start < day_end:
            total_time += day_end - day_start

        # Переходим к следующему дню
        current_day += timedelta(days=1)

    # Возвращаем общее время в секундах
    return total_time.total_seconds()

Мы создали необходимую функцию, перейдем к преобразованию данных

In [7]:
# Группировка сообщений в блоки по смене типа (клиент/менеджер)
# 1. Сортируем сообщения внутри каждой сделки по времени
df_chat_sorted = df_chat.sort_values(['entity_id', 'created_at'])

# 2. Определяем границы между блоками сообщений
# Создаём колонку с типом предыдущего сообщения в рамках сделки
df_chat_sorted['shifted_type'] = df_chat_sorted.groupby('entity_id')['type'].shift(1)

# 3. Помечаем начало новой группы, если тип сменился (клиент ↔ менеджер)
df_chat_sorted['is_new_group'] = (df_chat_sorted['type'] != df_chat_sorted['shifted_type']).astype(int)

# 4. Создаём идентификатор группы через кумулятивную сумму изменений
df_chat_sorted['group_id'] = df_chat_sorted.groupby('entity_id')['is_new_group'].cumsum()

# Выбор первого сообщения в каждой группе
# 5. Группируем по сделке и группе, берём первое сообщение из каждого блока
first_messages = df_chat_sorted.groupby(['entity_id', 'group_id']).first().reset_index()

# Разделение на входящие и исходящие сообщения
# 6. Фильтруем сообщения клиентов (входящие)
incoming = first_messages[first_messages['type'] == 'incoming_chat_message'].copy()

# 7. Фильтруем ответы менеджеров (исходящие)
outgoing = first_messages[first_messages['type'] == 'outgoing_chat_message'].copy()

# Сопоставление входящих и исходящих сообщений
# 8. Переименовываем колонки перед объединением
incoming_sorted = incoming.rename(columns={'created_at': 'client_time'}).sort_values('client_time')
outgoing_sorted = outgoing.rename(columns={'created_at': 'manager_time'}).sort_values('manager_time')

# 9. Объединяем с явным указанием колонок и фильтрацией NaN
paired = pd.merge_asof(
    incoming_sorted,
    outgoing_sorted,
    left_on='client_time',
    right_on='manager_time',
    by='entity_id',
    direction='forward'
).dropna(subset=['manager_time'])  # Удаляем строки без ответа

# Проверяем наличие данных после фильтрации
if paired.empty:
    raise ValueError("Нет данных для анализа: все ответы отсутствуют")

# 10. Теперь используем правильные названия колонок с проверкой типов
paired['response_seconds'] = paired.apply(
    lambda row: calculate_work_time(
        int(row['client_time']),  # Явное преобразование в int
        int(row['manager_time'])  # на случай float-значений
    ),
    axis=1
)

# Объединение с именами менеджеров и расчет среднего
# 11. Группируем по менеджерам и считаем среднее время ответа
result = paired.groupby('created_by_y')['response_seconds'].mean().reset_index()

# 12. Добавляем имена менеджеров из справочника
result = result.merge(
    df_managers, 
    left_on='created_by_y',  # ID менеджера из данных ответов
    right_on='mop_id'        # ID менеджера из справочника
)

# 13. Выводим финальный результат:
print(f'Длина итоговой таблицы = {len(result)}')
result.head()

Длина итоговой таблицы = 22


,created_by_y,response_seconds,mop_id,name_mop,rop_id
0,6645315.0,514.123810,6645315,Гюнель и Илина,1
1,6744792.0,565.912941,6744792,Юля и Наташа,1
2,6780177.0,237.420000,6780177,Даша и Даша,2
3,7408305.0,282.826087,7408305,Влада и Настя,3
4,7417617.0,502.214286,7417617,Порхачева Полина,3


Мы получили таблицу, добавим также столбец с именами начальников отделов продаж, соединив таблицу `result` с `test.rops` по `rop_id`

In [8]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   created_by_y      22 non-null     float64
 1   response_seconds  22 non-null     float64
 2   mop_id            22 non-null     int64  
 3   name_mop          22 non-null     object 
 4   rop_id            22 non-null     object 
dtypes: float64(2), int64(1), object(2)
memory usage: 1012.0+ bytes


Для соедиения прийдется исправить тип столбца `rop_id` на `odject`

In [9]:
result['rop_id'] = result['rop_id'].astype('int')
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   created_by_y      22 non-null     float64
 1   response_seconds  22 non-null     float64
 2   mop_id            22 non-null     int64  
 3   name_mop          22 non-null     object 
 4   rop_id            22 non-null     int32  
dtypes: float64(2), int32(1), int64(1), object(1)
memory usage: 924.0+ bytes


In [10]:
table = result.merge(df_rops, on='rop_id', how='left')
table.info()
table.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   created_by_y      22 non-null     float64
 1   response_seconds  22 non-null     float64
 2   mop_id            22 non-null     int64  
 3   name_mop          22 non-null     object 
 4   rop_id            22 non-null     int32  
 5   rop_name          22 non-null     object 
dtypes: float64(2), int32(1), int64(1), object(2)
memory usage: 1.1+ KB


,created_by_y,response_seconds,mop_id,name_mop,rop_id,rop_name
0,6645315.0,514.123810,6645315,Гюнель и Илина,1,Катя РОП
1,6744792.0,565.912941,6744792,Юля и Наташа,1,Катя РОП
2,6780177.0,237.420000,6780177,Даша и Даша,2,Полина РОП
3,7408305.0,282.826087,7408305,Влада и Настя,3,Эля РОП
4,7417617.0,502.214286,7417617,Порхачева Полина,3,Эля РОП


Данные можно успешно сохранять и переходить к дашборду.

In [11]:
#table.to_csv('test_table.csv', index=False)

## ССылка на дашборд

https://datalens.yandex/jeg0c9qcuo2a5